In [1]:
"""
When fixes are applied earlier in the chain, we don't want to have to re-do geocoding.
This file takes the latest version of the ODEF from the parsing step, and joins it on "ODEF_Geocoded_Valid"
"""
import pandas as pd

In [22]:
df = pd.read_csv("combined.csv", dtype="str", low_memory=False)
df_geo = pd.read_csv("ODHFv2_Geocoded_OSM_17-02-2022.csv", dtype="str", low_memory=False)

In [18]:
len(df_geo)

150

In [20]:
df.head()

,idx,filename,facility_name,facility_type,health_authority,alternative_name,number_beds,address_str,unit,street_no,...,province,postal_code,provider,latitude,longitude,geo_source,phone,email,website,dupe
0,00b7c41d6b9e6146d1817e6ef90f04,QC_healthcare_establishments.csv,Centre Le Cardinal inc.,NaN,RLS de Rivière-des-Prairies - Anjou - Montréal...,Centre Le Cardinal inc.,NaN,NaN,NaN,12900,...,QC,H1A1R9,Province of Quebec,45.6524505615,-73.4883346558,Source,NaN,NaN,NaN,False
1,d069e3165c6b92edc897f8d28ba75c,QC_healthcare_establishments.csv,Groupe Roy santé Inc.,NaN,RLS de Côte-des-Neiges - Métro - Parc-Extension,Groupe Roy santé Inc.,NaN,NaN,NaN,3550,...,QC,H2X4C5,Province of Quebec,45.5122146606,-73.5729370117,Source,NaN,NaN,NaN,False
2,cf496c9d20aeeb8b2fbf5c4b9feac3,QC_healthcare_establishments.csv,Centre régional de santé et de services sociau...,NaN,NaN,CRSSS de la Baie-James,NaN,NaN,NaN,312,...,QC,G8P1N5,Province of Quebec,49.913394928,-74.3570785522,Source,NaN,NaN,NaN,False
3,580165dcf73493eb41e0e1c8758f19,QC_healthcare_establishments.csv,Centre hospitalier de l'Université de Montréal,NaN,RLS des Faubourgs - Plateau-Mont-Royal - St-Lo...,CHUM,NaN,NaN,NaN,850,...,QC,H2X0A9,Province of Quebec,45.5108375549,-73.5559463501,Source,NaN,NaN,NaN,False
4,7f6887d2a41ad841ba7b6b0f399043,QC_healthcare_establishments.csv,CHSLD Résidence Bourg-Joli Inc.,NaN,RLS de Richelieu-Yamaska,CHSLD Résidence Bourg-Joli Inc.,NaN,NaN,NaN,2915,...,QC,J2S4Z3,Province of Quebec,45.6335144043,-72.964225769,Source,NaN,NaN,NaN,False


In [ ]:
# SL we have a unique idx so we don't need to deduplicate at this stage

# # merging doesn't work well with duplicates, so we'll do a simple deduplication here

# df_geo['dupe']=df_geo.duplicated(subset=['provider','source_id', 'facility_name','address_str','city'], keep=False)
# df['dupe']=df.duplicated(subset=['provider','source_id', 'facility_name','address_str','city'], keep=False)

# df.loc[df.dupe==True].to_csv('simple_duplicates_check.csv',index=False)
# df_geo.loc[df_geo.dupe==True].to_csv('simple_duplicates_geo_check.csv',index=False)

# df_geo=df_geo.drop_duplicates(subset=['provider','source_id', 'facility_name','address_str','city'])
# df=df.drop_duplicates(subset=['provider','source_id', 'facility_name','address_str'])

print(len(df_geo)-len(df))

#we will do it in two steps. First on records with source_ids, and then on those that don't.

df_source=df.copy()
df_none=df.copy()

df_source=df_source.loc[~df_source.source_id.isnull()]
df_none=df_none.loc[df_none.source_id.isnull()]

df1 = pd.merge(df_source, df_geo, on=['source_id', 'provider'], how='left', suffixes=('','_y'))
df2 = pd.merge(df_none, df_geo, on=['facility_name', 'provider','city','address_str'], how='left', suffixes=('','_y'))



print(len(df), len(df1)+len(df2))

DF=pd.concat([df1,df2])

DF['latitude']=DF['latitude_y']
DF['longitude']=DF['longitude_y']
DF['geo_source']=DF['geo_source_y']

#drop all duplicate columns

full_list=list(DF)
drop_list=[x for x in full_list if x.endswith('_y')]

DF=DF.drop(columns=drop_list)

DF.to_csv("output/ODEFv2_Geocoded_31-03-2021.csv",index=False)